#### import libs

In [ ]:
import cv2
from skimage.metrics import structural_similarity as ssim
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
from fpdf import FPDF
import os
import shutil
import imutils

#### hyper-parameters

In [ ]:
'''
frame_check_freq controls how frequent you want to capture the frames in video
example 30 dictates capturing every 30 frames
'''
frame_check_freq = 30
''' 
seconds_check_freq controls for how many second interval you want to capture the frames in video
example 2 dictates capturing frame every 2 seconds

'''
seconds_check_freq = 2
# increase these 2 hyper-param if the video haults on single frame for longer duration

#### config

In [ ]:
video_path = "./data/lecture6.mp4"

#### video summary

In [ ]:
video_cap = cv2.VideoCapture(video_path)

# count the number of frames 
frames = video_cap.get(cv2.CAP_PROP_FRAME_COUNT) 
fps = video_cap.get(cv2.CAP_PROP_FPS) 
  
# calculate duration of the video 
seconds = round(frames / fps) 
video_time = datetime.timedelta(seconds=seconds) 
print(f"duration in seconds: {seconds}") 
print(f"video time: {video_time}") 
print(f"Total video frames: {frames} and fps: {fps}")

video_cap.release()

#### custom function to check image similarity

In [ ]:
def is_similar_image(img1, img2, similarity_thresh=0.8):
    img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    similarity_score = ssim(img1, img2)
    if similarity_score >= similarity_thresh:
        return True, similarity_score
    else:
        return False, similarity_score

In [ ]:
img1_2 = cv2.imread("test/img1_2.png")
img1_2_shape = img1_2.shape
img2 = cv2.resize(cv2.imread("test/img2.png"), (img1_2_shape[1], img1_2_shape[0]))
img1 = cv2.resize(cv2.imread("test/img1.png"), (img1_2_shape[1], img1_2_shape[0]))
print(img1_2_shape, img1.shape, img2.shape)
print(is_similar_image(img1, img1_2, similarity_thresh=0.8))
print(is_similar_image(img1, img2, similarity_thresh=0.8))

#### extract frame based on hyper-parameters

In [ ]:
print(f"Video fps: {fps}")
if seconds_check_freq == 1 and fps >= frame_check_freq and fps % frame_check_freq == 0:
    print(f"Capturing {int(fps/frame_check_freq)} frames per second")
elif seconds_check_freq > 1:
    print(f"Capturing a frame every {seconds_check_freq} seconds")
else:
    print(f"Capturing every {frame_check_freq} alternate frame")


captured_frames = []
distinct_images = []

frame_count = 0
frame_read_count = 0
ref_frame = None
frame_select_index = seconds_check_freq*frame_check_freq
video_cap = cv2.VideoCapture(video_path)
while(video_cap.isOpened()): 
    ret, frame = video_cap.read() 
    if isinstance(ref_frame, type(None)):
        ref_frame = frame
        distinct_images.append(ref_frame[:])
    if ret == True: 
        frame_count += 1
        if frame_count % frame_select_index == 0:
            frame_read_count += 1
            is_similar, similarity_score = is_similar_image(frame, ref_frame, 0.8)
            if not is_similar:
                distinct_images.append(frame)
                ref_frame = frame
    else: 
        break
    print(f"Captured {frame_read_count} out of {int(frames/(2*frame_check_freq))} frames. Captured {len(distinct_images)} distinct frames")

print(f"parsed frames: {frame_count}")
print(f"selected frames: {len(distinct_images)}")
video_cap.release() 

In [ ]:
filtered_slides = distinct_images

#### filter distinct images based on similarity scores

In [ ]:
distinct_images = []

cur_ref_image = captured_frames[-1]
distinct_images.append(cur_ref_image)
for image in tqdm(captured_frames[::-1]):
    is_similar, similarity_score = is_similar_image(cur_ref_image, image, 0.6)
    if not is_similar:
        distinct_images.append(image)
        cur_ref_image = image

filtered_slides = list(reversed(distinct_images))
print(f"Found {len(distinct_images)} frames")

#### export images

In [ ]:
output_frame_dir = f"{os.getcwd()}/slide_images/"
if not os.path.exists(output_frame_dir):
    os.makedirs(output_frame_dir)
else:
    shutil.rmtree(output_frame_dir)
    os.makedirs(output_frame_dir)

img_h, img_w = filtered_slides[0].shape[:-1]
pdf_w, pdf_h = float(img_w * 0.264583), float(img_h * 0.264583)
pdf_w_limit, pdf_h_limit = 297, 210
for slide_idx, slide_image in tqdm(enumerate(filtered_slides)):
    # image resizing to make sure image dim doesn't exceed pdf dim
    if pdf_w > pdf_h and pdf_w > pdf_w_limit:
        slide_image = imutils.resize(slide_image, pdf_w_limit)
        pdf_w, pdf_h = pdf_w_limit, slide_image.shape[0]
    elif pdf_h > pdf_h_limit:
        slide_image = imutils.resize(slide_image, pdf_h_limit)
        pdf_h, pdf_w = pdf_h_limit, slide_image.shape[1]
    cv2.imwrite(f"{output_frame_dir}slide_{slide_idx}.jpg", slide_image)

#### repeatitive images can be discarded now from the export folder

#### export to pdf

In [ ]:
video_name_without_extension = ".".join(video_path.split("/")[-1].split(".")[:-1])
pdf_name = f"{video_name_without_extension}.pdf"
print(pdf_name)

if pdf_h > pdf_w:
    pdf = FPDF(unit = "mm", format = "A4")
else:
    pdf = FPDF("L", unit = "mm", format = "A4")

# imagelist is the list with all image filenames
for image_idx in range(len(os.listdir(output_frame_dir))):
    image_path = f"{output_frame_dir}slide_{image_idx}.jpg"
    pdf.add_page()
    pdf.image(image_path, x=0, y=0, w=pdf_w, h=pdf_h)
pdf.output(pdf_name, "F")